In [1]:
import os
import time
import math
import random
import itertools
import numpy as np
import pandas as pd
# PyTorch
import torch
import torchvision
import torchmetrics

In [2]:
import sys
sys.path.append('../src/')

%load_ext autoreload
%autoreload 2
# Importing our custom module(s)
import losses
import utils

In [6]:
dataset = 'CIFAR-10'
tuned_df = pd.read_csv(f'/cluster/home/eharve06/informed-priors/notebooks/tuned_{dataset}.csv')
retrained_df = pd.read_csv(f'/cluster/home/eharve06/informed-priors/notebooks/retrained_{dataset}.csv')
tuned_df.head()

,avg_sec/epoch,method,model_name,n,random_state,runtime,val_acc,val_nll
0,0.262689,l2-sp,l2-sp_lr_0=0.1_n=100_random_state=1001_tau=0.01,100,1001,1576.135183,0.45,1.874081
1,0.247174,l2-sp,l2-sp_lr_0=0.1_n=100_random_state=1001_tau=0.001,100,1001,1483.043704,0.20,6.743123
2,0.246002,l2-sp,l2-sp_lr_0=0.1_n=100_random_state=1001_tau=0.0001,100,1001,1476.009241,0.20,6.849158
3,0.244324,l2-sp,l2-sp_lr_0=0.1_n=100_random_state=1001_tau=1e-05,100,1001,1465.942561,0.05,10.149165
4,0.244776,l2-sp,l2-sp_lr_0=0.1_n=100_random_state=1001_tau=1e-06,100,1001,1468.657881,0.15,12.602262


In [17]:
def format_time(seconds):
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return f"{hours} hrs. {minutes} mins. {seconds} secs."

In [28]:
condition = (tuned_df.n==50000)&(tuned_df.method=='l2-sp')&(tuned_df.random_state==1001)
temp_df = tuned_df.loc[condition]
total_time = temp_df.runtime.sum()
best_model_name = temp_df.loc[temp_df.val_nll.idxmin(), 'model_name']
test_acc = retrained_df.loc[retrained_df.model_name==best_model_name, 'test_acc'].item()
total_time += retrained_df.loc[retrained_df.model_name==best_model_name, 'runtime'].item()
print(f'Average runtime (sec.): {temp_df.runtime.mean()}')
print(f'Total time (sec.): {total_time}')

Average runtime (sec.): 2351.4882265925407
Total time (sec.): 58555.35243535042


In [29]:
condition = (tuned_df.n==50000)&(tuned_df.method=='ptyl')&(tuned_df.random_state==1001)
temp_df = tuned_df.loc[condition]
total_time = temp_df.runtime.sum()
best_model_name = temp_df.loc[temp_df.val_nll.idxmin(), 'model_name']
test_acc = retrained_df.loc[retrained_df.model_name==best_model_name, 'test_acc'].item()
total_time += retrained_df.loc[retrained_df.model_name==best_model_name, 'runtime'].item()
print(f'Average runtime (sec.): {temp_df.runtime.mean()}')
print(f'Total time (sec.): {total_time}')

Average runtime (sec.): 2234.773100433747
Total time (sec.): 538565.2564959526


In [22]:
experiments_directory = '/cluster/tufts/hugheslab/eharve06/informed-priors/experiments/retrained_CIFAR-10_VI'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
methods = ['l2-zero', 'l2-sp', 'ptyl']
ns = [100, 1000, 10000, 50000]
random_states = [1001, 2001, 3001]

columns = ['avg_sec/epoch', 'method', 'model_name', 'n', 'random_state', 'runtime', 'train_loss', 'test_acc', 'test_nll']
results_df = pd.DataFrame(columns=columns)

for lr_0, method, n, random_state in itertools.product(lr_0s, methods, ns, random_states):
    model_name = f'{method}_kappa={23528522/n}_lr_0={lr_0}_n={n}_random_state={random_state}'
    temp_df = pd.read_csv(f'{experiments_directory}/{model_name}.csv')
    row = [temp_df['train_sec/epoch'].mean(), method, model_name, n, random_state, temp_df['train_sec/epoch'].sum(), temp_df.train_loss.values[-1], temp_df.val_or_test_acc.values[-1], temp_df.val_or_test_nll.values[-1]]
    results_df.loc[len(results_df)] = row

In [25]:
condition = (results_df.n==50000)&(results_df.method=='l2-sp')&(results_df.random_state==1001)
temp_df = results_df.loc[condition]
total_time = temp_df.runtime.sum()
print(f'Average runtime (sec.): {temp_df.runtime.mean()}')
print(f'Total time (sec.): {total_time}')

Average runtime (sec.): 0.0 hrs. 39.0 mins. 0.16479343175888062 secs.
Total time (sec.): 2.0 hrs. 36.0 mins. 0.6591737270355225 secs.


In [27]:
condition = (results_df.n==50000)&(results_df.method=='ptyl')&(results_df.random_state==1001)
temp_df = results_df.loc[condition]
total_time = temp_df.runtime.sum()
print(f'Average runtime (sec.): {temp_df.runtime.mean()}')
print(f'Total time (sec.): {total_time}')

Average runtime (sec.): 2399.5411801338196
Total time (sec.): 9598.164720535278
